In [2]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, CONNECTIONS_CONFIG, QWEN_1_5_B

cfg = CONNECTIONS_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = QWEN_1_5_B
cfg.notebook_mode = True
cfg.max_samples = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-04-12 07:59:12.656 | INFO     | trl_wrapper.trainer_wrapper:__init__:285 - Setting padding side to: left
2025-04-12 07:59:12.673 | INFO     | trl_wrapper.trainer_wrapper:init_model:296 - Using device: mps
2025-04-12 07:59:12.673 | INFO     | trl_wrapper.trainer_wrapper:init_model:310 - Loading model Qwen/Qwen2.5-1.5B-Instruct with attn_impl: sdpa
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
from model.reasoning import ConnectionsDataModule

In [5]:
conn = ConnectionsDataModule(wrapper.tokenizer, wrapper.config)

2025-04-12 07:59:14.888 | INFO     | trl_wrapper.wrapper_config:__init__:145 - Cache dir: ../dataset_caches/connections_data_module


In [6]:
wrapper.init_data_module()

2025-04-12 07:59:14.928 | INFO     | trl_wrapper.wrapper_config:__init__:145 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 100000/100000 [00:05<00:00, 19801.26 examples/s]


In [7]:
wrapper.init_trainer()

2025-04-12 07:59:36.834 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:373 - Saving output to: ./runs/04-12-7-59-505124-qwen2.5-1.5b-instruct-
2025-04-12 07:59:36.835 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:396 - Initializing trainer, run_name: 04-12-7-59-505124-qwen2.5-1.5b-instruct-, wandb project: qwen-connections-grpo
2025-04-12 07:59:36.835 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:479 - Using vllm: False


In [ ]:
wrapper.train()

2025-04-12 09:33:25.835 | INFO     | trl_wrapper.trainer_wrapper:train:644 - Starting training.
